# read data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# missing values over years (optional)

In [111]:
qtw = pd.read_csv("QTW14-19.csv")
missing_values_over_yrs = qtw.drop('Year', axis = 1).isna().groupby(qtw.Year, sort=False).sum().reset_index() / len(qtw) * 100 > (100/12)

In [135]:
qtw.groupby('Treatment').count()

,Unnamed: 0,Year,Group,Serial_Num,Age,Age_5Gp,Sex,District_living,Housing,Employment,...,TrajP_CC_Gp4,TrajP_CCA_Gp1,TrajP_CCA_Gp2,TrajP_CCA_Gp3,TrajP_CCA_Gp4,Group_traj_ITT,Group_traj_CC,Group_traj_CCA,Arm,ID
Treatment,,,,,,,,,,,,,,,,,,,,,
0,2896,2896,2896,2896,1810,2724,2895,2549,2352,2467,...,2896,2896,2896,2896,2896,2896,2896,2896,2896,2896
1,3692,3692,3692,3692,2591,3517,3692,3331,3105,3228,...,3692,3692,3692,3692,3692,3692,3692,3692,3692,3692


In [112]:
missing_values_over_yrs

,Year,Unnamed: 0,Group,Serial_Num,Age,Age_5Gp,Sex,District_living,Housing,Employment,...,TrajP_CCA_Gp1,TrajP_CCA_Gp2,TrajP_CCA_Gp3,TrajP_CCA_Gp4,Group_traj_ITT,Group_traj_CC,Group_traj_CCA,Arm,Treatment,ID
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [77]:
missing_values_over_yrs.to_excel("missing_values_over_yrs.xlsx")

In [34]:
# drop cols with > .5 missing values in one of the years
qtw = qtw.drop(['Age', 
                'Marital',
                'Income_5Gp',
                'Base_QRMethod_1',
                'Base_QRMethod_10',
                'Base_QRMethod_11',
                'Base_QRMethod_12',
                'Base_QRMethod_13',
                'M1_QReason_6',
                'M1_QReason_13',
                'ID',
                'Serial_Num',
                'Unnamed: 0'], axis=1)

# one-hot encoded qtw dataframe

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
qtw = pd.read_csv("QTW14-19.csv")
# drop cols with > .5 missing values
qtw = qtw.drop(['Children', 
                'Children_coliving',
                'Income_since18',
                'Base_QRMethod_5',
                'Base_QRMethod_14',
                'Base_QRMethod_15',
                'Base_QRMethod_16',
                'M1_QA_acc',
                'M1_QReason_3',
                'M1_QReason_8'], axis=1)
# drop traj cols
qtw = qtw[qtw.columns.drop(list(qtw.filter(regex='Traj')))]
qtw = qtw[qtw.columns.drop(list(qtw.filter(regex='traj')))]
# drop cols with > .5 missing values in one of the years
qtw = qtw.drop(['Age', 
                'Marital',
                'Income_5Gp',
                'Base_QRMethod_1',
                'Base_QRMethod_10',
                'Base_QRMethod_11',
                'Base_QRMethod_12',
                'Base_QRMethod_13',
                'M1_QReason_6',
                'M1_QReason_13',
                'ID',
                'Serial_Num',
                'Unnamed: 0'], axis=1)
# remove ITT columns except M6_VQ_ITT
m6_vq_itt = qtw["M6_VQ_ITT"]
qtw = qtw[qtw.columns.drop(list(qtw.filter(regex='M6_')))]
qtw = qtw[qtw.columns.drop(list(qtw.filter(regex='_ITT')))]
qtw["M6_VQ_ITT"] = m6_vq_itt
# remove _QA cols
# qtw = qtw[qtw.columns.drop(list(qtw.filter(regex='_QA')))]
# remove year, arm and treatment cols
qtw = qtw.drop(['Year', 'Arm', 'Treatment','Group'], axis = 1)

/Users/ytf1997/miniconda3/envs/ML_learn/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (3,12,15,42,43,44,49) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
len(qtw.columns.values)

67

# KNN imputation missing data

In [6]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
import math

df = qtw

mm = MinMaxScaler()

def find_category_mappings(df, variable):
    return {k: i for i, k in enumerate(df[variable].dropna().unique(), 0)}

def integer_encode(df , variable, ordinal_mapping):
    df[variable] = df[variable].map(ordinal_mapping)

mappin = dict()
def imputation(df1 , cols):
    df = df1.copy()
    #Encoding dict &amp; Removing nan    
    #mappin = dict()
    for variable in cols:
        mappings = find_category_mappings(df, variable)
        mappin[variable] = mappings

    #Apply mapping
    for variable in cols:
        integer_encode(df, variable, mappin[variable])  

    #Minmaxscaler and KNN imputation 
    sca = mm.fit_transform(df)
    knn_imputer = KNNImputer(n_neighbors = int(math.sqrt(len(df))))
    knn = knn_imputer.fit_transform(sca)
    df.iloc[:,:] = mm.inverse_transform(knn)
    for i in df.columns : 
        df[i] = round(df[i]).astype('int')

    #Inverse transform
    for i in cols:
        inv_map = {v: k for k, v in mappin[i].items()}
        df[i] = df[i].map(inv_map)
    return df


#Imputation
qtw = imputation(qtw,qtw.select_dtypes(include = "object").columns.values.tolist())

# VIF for cont. variables

In [7]:
cont_colnames = qtw.select_dtypes(exclude = "object").columns.values.tolist()
qtw_cont = qtw[cont_colnames]

In [8]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif_data = pd.DataFrame()
vif_data["feature"] = qtw_cont.columns
vif_data["VIF"] = [variance_inflation_factor(qtw_cont.values, i)
                          for i in range(len(qtw_cont.columns))]

/Users/ytf1997/miniconda3/envs/ML_learn/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:1717: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.uncentered_tss


In [9]:
vif_data.to_excel("vifv3.xlsx")

In [10]:
drop_names = vif_data[vif_data["VIF"] >= 10]["feature"].tolist()
qtw = qtw[qtw.columns.drop(drop_names)]

In [11]:
drop_names

['Base_QImportance',
 'Base_QDifficulty',
 'M3_QImportance',
 'M3_QDifficulty',
 'M3_QConfidence']

In [12]:
len(qtw.columns.values)

62

# Cat. feature selection
By chi-sq. test, only keep features that have p-value < .05

In [13]:
cat_colnames = qtw.select_dtypes(include = "object").columns.values.tolist()
qtw_cat = qtw[cat_colnames]

In [14]:
# Import the function
from scipy.stats import chi2_contingency
#Testing the relationship

chi2_check = []
categorical_columns = qtw_cat.columns.values.tolist()
for i in categorical_columns:
    if chi2_contingency(pd.crosstab(qtw_cat['M6_VQ_ITT'], qtw_cat[i]))[1] < 0.05:
        chi2_check.append('Reject Null Hypothesis')
    else:
        chi2_check.append('Fail to Reject Null Hypothesis')
res = pd.DataFrame(data = [categorical_columns, chi2_check] 
             ).T 
res.columns = ['Column', 'Hypothesis']

In [15]:
print(res[res['Hypothesis'] == "Fail to Reject Null Hypothesis"]['Column'].tolist())

['Sex', 'Base_QRMethod_2', 'Base_QRMethod_3', 'Base_QRMethod_4', 'Base_QRMethod_6', 'Base_QRMethod_7', 'Base_QRMethod_8']


In [16]:
res.to_excel("cat_variables.xlsx")

In [17]:
drop_names = res[res['Hypothesis'] == "Fail to Reject Null Hypothesis"]['Column'].tolist()
qtw = qtw[qtw.columns.drop(drop_names)]

In [18]:
len(qtw.columns.values)

55

# LASSO

In [19]:
qtw_encoded = pd.get_dummies(qtw, drop_first = True)

In [20]:
qtw.columns.values

array(['Age_5Gp', 'District_living', 'Housing', 'Employment', 'Edu',
       'Base_CO', 'Base_Time_1st_cig', 'Base_HSI', 'Base_CpdGp',
       'Base_Cpd', 'Base_Age_weekly_smoking', 'Base_QA', 'Base_QA_time',
       'Base_Ready', 'Base_QConfidence', 'M0_R_P_c', 'Base_QRMethod_9',
       'M1_Q', 'M1_Cpd', 'M1_QA', 'M1_QA_Num', 'M1_R50', 'M1_R_P_c',
       'M1_QReason_1', 'M1_QReason_2', 'M1_QReason_4', 'M1_QReason_5',
       'M1_QReason_7', 'M1_QReason_9', 'M1_QReason_10', 'M1_QReason_11',
       'M1_QReason_12', 'M1_Withdrawal_1', 'M1_Withdrawal_2',
       'M1_Withdrawal_3', 'M1_Withdrawal_4', 'M1_Withdrawal_5',
       'M1_Withdrawal_6', 'M1_Withdrawal_7', 'M1_Withdrawal_8',
       'M1_Withdrawal_9', 'M1_Withdrawal_10', 'M2_Q', 'M2_Cpd', 'M2_QA',
       'M2_QA_acc', 'M2_R50', 'M2_R_P_c', 'M3_Q', 'M3_Cpd', 'M3_QA',
       'M3_QA_acc', 'M3_R50', 'M3_R_P_c', 'M6_VQ_ITT'], dtype=object)

In [21]:
from sklearn.linear_model import LogisticRegression
X = qtw_encoded[qtw_encoded.columns.drop(list(qtw_encoded.filter(regex='M6_VQ_ITT')))]
y = qtw_encoded["M6_VQ_ITT_Smoking"]
y = 1 - y
y.rename("M6_VQ_ITT_Abstinence")

0       1
1       0
2       0
3       0
4       0
       ..
6583    0
6584    0
6585    1
6586    0
6587    0
Name: M6_VQ_ITT_Abstinence, Length: 6588, dtype: uint8

In [22]:
log_reg = LogisticRegression(penalty='l1', solver='liblinear').fit(X, y)

In [23]:
lasso_df = pd.DataFrame({"Features": X.columns.values.tolist(),
                    "Coef": log_reg.coef_[0].tolist()})

In [24]:
lasso_df.to_excel("lasso.xlsx")
lasso_df[lasso_df["Coef"] == 0].to_excel("lasso_dump.xlsx")
lasso_df[lasso_df["Coef"] != 0].to_excel("lasso_subset.xlsx")

In [25]:
print(lasso_df[lasso_df["Coef"] == 0]["Features"].tolist())

['M0_R_P_c', 'District_living_Kowloon City', 'District_living_Kwai Tsing', 'District_living_Mainland', 'District_living_Shatin', 'District_living_Wong Tai Sin', 'Base_CpdGp_31+', 'Base_QA_time_within 6 months (but before the passing month)', 'Base_Ready_In 60 days', 'Base_QRMethod_9_Use', 'M1_QA_Num_Twice', 'M1_QReason_4_No quit attempt after joining QTW', 'M1_QReason_5_No quit attempt after joining QTW', 'M1_QReason_5_Yes', 'M1_QReason_9_No quit attempt after joining QTW', 'M1_QReason_10_No quit attempt after joining QTW', 'M1_QReason_11_No quit attempt after joining QTW', 'M1_Withdrawal_1_Yes', 'M1_Withdrawal_3_No quit attempt after joining QTW', 'M1_Withdrawal_4_No quit attempt after joining QTW', 'M1_Withdrawal_4_Yes', 'M1_Withdrawal_5_No quit attempt after joining QTW', 'M1_Withdrawal_7_Yes', 'M2_QA_Yes']


In [177]:
drop_names = [
    "M0_R_P_c",
    "M1_R_P_c",
    "M2_R_P_c",
    "M3_R_P_c",
    "Edu",
    "Base_Time_1st_cig",
    "Base_QA",
    "Base_QRMethod_9",
    "M1_QA",
    "M1_QReason_2",
    "M1_QReason_5",
    "M1_QReason_7",
    "M1_QReason_10",
    "M1_QReason_11",
    "M1_Withdrawal_1",
    "M1_Withdrawal_2",
    "M1_Withdrawal_4",
    "M1_Withdrawal_6",
    "M1_Withdrawal_7",
    "M1_Withdrawal_9",
    "M3_QA_acc"
]
qtw = qtw[qtw.columns.drop(drop_names)]

# save variables to pickle

In [178]:
select_columns = qtw.columns.values.tolist()
qtw_orig = pd.read_csv("QTW14-19.csv")

qtw_subset = qtw_orig[select_columns]

qtw_subset = qtw_subset[qtw_subset.columns.drop(list(qtw.filter(regex='M6_')))]
qtw_subset["M6_VQ_ITT"] = qtw_orig["M6_VQ_ITT"]

/Users/ytf1997/miniconda3/envs/ML_learn/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (3,12,15,42,43,44,49) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [180]:
qtw_subset.columns.values.tolist()

['Age_5Gp',
 'Housing',
 'Employment',
 'Base_CO',
 'Base_HSI',
 'Base_CpdGp',
 'Base_Cpd',
 'Base_Age_weekly_smoking',
 'Base_QA_time',
 'Base_Ready',
 'Base_QConfidence',
 'Base_QRMethod_6',
 'M1_Q',
 'M1_Cpd',
 'M1_QA_Num',
 'M1_R50',
 'M1_QReason_1',
 'M1_QReason_4',
 'M1_QReason_9',
 'M1_QReason_12',
 'M1_Withdrawal_3',
 'M1_Withdrawal_5',
 'M1_Withdrawal_8',
 'M1_Withdrawal_10',
 'M2_Q',
 'M2_Cpd',
 'M2_QA',
 'M2_QA_acc',
 'M2_R50',
 'M3_Q',
 'M3_Cpd',
 'M3_QA',
 'M3_R50',
 'M6_VQ_ITT']

In [181]:
# save variables to pickle
import pickle
filename = "variables.pickle"
with open(filename, 'wb') as f:
    pickle.dump({
        'qtw': qtw_subset
    }, f)

# get variables from pickle

In [25]:
import pickle
import pandas as pd

with open('variables.pickle', 'rb') as f:
    pic = pickle.load(f)
# qtw_imputed = pic['qtw_imputed']
qtw_encoded = pic['qtw_encoded']